In [28]:
import os
import json
import requests
import pandas as pd

In [29]:
google_place = os.environ["GOOGLE_API"]

In [30]:
lat = ' 49.246292'
lon = ' -123.116226'

In [31]:
import time

# Define API key and endpoint
url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

# Define the parameters for request
params = {
    'query': 'restaurants in Vancouver',
    'radius': 1000,  # Radius in meters
    'key': google_place
}

# Send the initial request
response = requests.get(url, params=params)

# Parse the response as JSON
data = response.json()

# Initialize an empty list to store restaurant details
restaurant_details = []

# Handle the response data
if response.status_code == 200:
    results = data['results']
    
    for result in results:
        name = result['name']
        address = result['formatted_address']
        rating = result.get('rating', 'N/A')
        place_id = result['place_id']
        location = result['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']
        
        restaurant_details.append({
            'Name': name,
            'Address': address,
            'Rating': rating,
            'Place ID': place_id,
            'Latitude': latitude,
            'Longitude': longitude
        })

    # Check if there are more results and retrieve them using pagination
    while 'next_page_token' in data:
        next_page_token = data['next_page_token']
        params['pagetoken'] = next_page_token
        
        # allow the next page token to become valid
        time.sleep(3)
        
        # Send the next page request
        response = requests.get(url, params=params)
        
        # Parse the response as JSON
        data = response.json()
        
        if response.status_code == 200:
            results = data['results']
            
            for result in results:
                name = result['name']
                address = result['formatted_address']
                rating = result.get('rating', 'N/A')
                place_id = result['place_id']
                location = result['geometry']['location']
                latitude = location['lat']
                longitude = location['lng']
                
                restaurant_details.append({
                    'Name': name,
                    'Address': address,
                    'Rating': rating,
                    'Place ID': place_id,
                    'Latitude': latitude,
                    'Longitude': longitude
                })
        else:
            print('Request failed with status code:', response.status_code)
else:
    print('Request failed with status code:', response.status_code)

# Create a DataFrame from the restaurant details list
google_rest = pd.DataFrame(restaurant_details)

# the DataFrame
#google_rest

In [32]:
google_rest.drop_duplicates(inplace=True)

In [33]:
google_rest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       60 non-null     object 
 1   Address    60 non-null     object 
 2   Rating     60 non-null     float64
 3   Place ID   60 non-null     object 
 4   Latitude   60 non-null     float64
 5   Longitude  60 non-null     float64
dtypes: float64(3), object(3)
memory usage: 3.3+ KB


In [34]:
google_rest.drop(['Place ID'], axis=1, inplace=True)

In [35]:
#google_rest

In [36]:
#Reset index:
google_rest.reset_index(drop=True, inplace=True)

In [37]:
#google_rest

In [38]:
google_rest['station_location'] = google_rest.apply(lambda row: f"{row['Latitude']:.6f}, {row['Longitude']:.6f}", axis=1)
google_rest

,Name,Address,Rating,Latitude,Longitude,station_location
0,Chambar Restaurant,"568 Beatty St, Vancouver, BC V6B 2L3, Canada",4.5,49.280127,-123.109935,"49.280127, -123.109935"
1,Cardero's Restaurant,"1583 Coal Harbour Quay, Vancouver, BC V6G 3E7,...",4.5,49.291610,-123.127482,"49.291610, -123.127482"
2,Elisa,"1109 Hamilton St, Vancouver, BC V6B 5P6, Canada",4.5,49.275964,-123.121413,"49.275964, -123.121413"
3,Giardino Restaurant,"1328 Hornby St, Vancouver, BC V6Z 1W5, Canada",4.6,49.276941,-123.129993,"49.276941, -123.129993"
4,Nightingale,"1017 W Hastings St, Vancouver, BC V6E 0C4, Canada",4.5,49.287398,-123.117475,"49.287398, -123.117475"
5,Joe Fortes Seafood & Chop House,"777 Thurlow St, Vancouver, BC V6E 3V5, Canada",4.5,49.284949,-123.124553,"49.284949, -123.124553"
6,Miku Vancouver,"200 Granville St #70, Vancouver, BC V6C 1S4, C...",4.5,49.287018,-123.113011,"49.287018, -123.113011"
7,Glowbal,"590 W Georgia St, Vancouver, BC V6E 1A3, Canada",4.3,49.281531,-123.117398,"49.281531, -123.117398"
8,PiDGiN Restaurant,"350 Carrall St, Vancouver, BC V6B 2J3, Canada",4.4,49.281881,-123.104145,"49.281881, -123.104145"
9,Hawksworth Restaurant,"801 W Georgia St, Vancouver, BC V6C 1P7, Canada",4.4,49.283453,-123.119239,"49.283453, -123.119239"


In [39]:
google_rplace = google_rest.to_csv('google_rplace.csv', index=False)

In [40]:
google_rplace = pd.read_csv('google_rplace.csv')